The data from the H1 radio telescope is stored in binary form, as spectra which are concatenated within the file.

To read these we'll need a bit of logic which will reshape the data into something we can perform analysis on.

Binary data can be read using the `scipy.fromfile` function.

In [10]:
import scipy
import numpy as np

All of the archival data for the telescope is available on the `cassegrain` system.
Let's load an example from the 2018 observing campaign.

In [15]:
datafile = "/data/cassegrain/srt/archive/srt2018/ra0.00dec10.00time2018-01-30T16:37:00.000.dat"

The filename for each observation contains metadata which we can use to determine the sky location of each observation.

Using regular expressions we can parse this data, so that we can use it later.

In [16]:
def extract_metadata(filename):
    """
    Extract all of the meta data contained within the name of a data file.
    
    Parameters
    ----------
    filename : str
        The name of the file, can optionally include the full path, 
        in which case the name is taken to be everything after the final
        slash.
        
    Returns
    -------
    position : `astropy.skycoord`
        The sky position at which the file starts
    time : `astropy.Time`
        The time at which the file starts.
    """
    
    a = re.compile("ra(.*)dec(.*)time(.*).dat")
    filepath = filename.split('/')

    s_ra = float(a.search(filepath[-1]).group(1))
    s_dec = float(a.search(filepath[-1]).group(2))
    s_time = a.search(filepath[-1]).group(3)
    
    
    time = Time(s_time, format='isot', scale='utc')
    position = SkyCoord(s_ra*u.deg, s_dec*u.deg, frame=ICRS, obstime = time)
    return position, time

We can then use this extracted meta data to prepare the data, which includes "rolling" the array so that the first entry corresponds to an RA of 0.

In [17]:
data = scipy.fromfile(datafile, dtype=np.float32)
position, time = extract_metadata(datafile)

data = np.reshape(data, [-1,256])

roll = int((position.ra/360).value * len(data)/256.0)
data = np.roll(data, roll, axis=0)

FileNotFoundError: [Errno 2] No such file or directory: '/data/cassegrain/srt/archive/srt2018/ra0.00dec10.00time2018-01-30T16:25:00.000.dat'